## Part 1: Preprocessing

In [50]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense
#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [51]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [52]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [53]:


# Create a list of at least 10 column names to use as X data
X_columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']
# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
print(X_df)





      Age  DistanceFromHome  Education  EnvironmentSatisfaction  HourlyRate  \
0      41                 1          2                        2          94   
1      49                 8          1                        3          61   
2      37                 2          2                        4          92   
3      33                 3          4                        4          56   
4      27                 2          1                        1          40   
...   ...               ...        ...                      ...         ...   
1465   36                23          2                        3          41   
1466   39                 6          1                        4          42   
1467   27                 4          3                        2          87   
1468   49                 2          3                        4          63   
1469   34                 8          3                        2          82   

      JobInvolvement  JobLevel  JobSatisfaction Mar

In [54]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [55]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
combined_data = pd.concat([X_train, X_test])
le = LabelEncoder()
for column in combined_data.columns:
    combined_data[column] = le.fit_transform(combined_data[column])
X_train = combined_data[:len(X_train)]
X_test = combined_data[len(X_train):]

In [56]:
# Create a StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [57]:
# Create a OneHotEncoder for the Department column
ohe = OneHotEncoder(sparse=False)
y_train_department = ohe.fit_transform(y_train[['Department']])
y_test_department = ohe.transform(y_test[['Department']])

# Create a OneHotEncoder for the Attrition column
ohe = OneHotEncoder(sparse=False)
y_train_attrition = ohe.fit_transform(y_train[['Attrition']])
y_test_attrition = ohe.transform(y_test[['Attrition']])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Create, Compile, and Train the Model

In [58]:
# Find the number of columns in the X training data
input_shape = X_train_scaled.shape[1]

# Create the input layer
input_layer = Input(shape=(input_shape,))

# Create at least two shared layers
shared_layer1 = Dense(64, activation='relu')(input_layer)
shared_layer2 = Dense(32, activation='relu')(shared_layer1)

In [59]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = Dense(32, activation='relu')(shared_layer2)

# Create the output layer
department_output = Dense(3, activation='softmax')(department_hidden)


In [62]:
# Create the output layer for department
department_output = Dense(3, activation='softmax')(shared_layer2)

# Create the output layer for attrition
attrition_output = Dense(2, activation='softmax')(shared_layer2)

# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer='adam', metrics=['accuracy', 'accuracy'])

# Summarize the model
print(model.summary())

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 22)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 64)             │          1,472 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 32)             │          2,080 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 3)              │             99 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 2)              │             66 │ dense_7[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,717 (14.52 KB)

 Trainable params: 3,717 (14.52 KB)

 Non-trainable params: 0 (0.00 B)

None


In [63]:
# Train the model
model.fit(X_train_scaled, [y_train_department, y_train_attrition], epochs=50, batch_size=32)

Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - dense_12_accuracy: 0.5327 - dense_13_accuracy: 0.6933 - loss: 1.5439
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_12_accuracy: 0.6554 - dense_13_accuracy: 0.8352 - loss: 1.2424
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_12_accuracy: 0.6450 - dense_13_accuracy: 0.8181 - loss: 1.1941
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_12_accuracy: 0.6588 - dense_13_accuracy: 0.8128 - loss: 1.1294
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_12_accuracy: 0.6350 - dense_13_accuracy: 0.8560 - loss: 1.0969
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_12_accuracy: 0.6609 - dense_13_accuracy: 0.8608 - loss: 1.0496
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_12_accuracy: 0.6778 - dense_13_accuracy: 0.8625 - loss: 1.0200
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_12_accuracy: 0.6749 - dense_13_accuracy: 0.8675 - loss: 1.0244
Epoch 9/50
37/37 ━━━━━━━━━━━━━━━

In [64]:
# Evaluate the model with the testing data
loss, department_accuracy, attrition_accuracy = model.evaluate(X_test_scaled, [y_test_department, y_test_attrition])

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - dense_12_accuracy: 0.5534 - dense_13_accuracy: 0.8435 - loss: 1.8147  


In [65]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {department_accuracy}, Attrition Accuracy: {attrition_accuracy}")

Department Accuracy: 0.5986394286155701, Attrition Accuracy: 0.8367347121238708


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. It is not the best it could have high accuracy simply a company has low attrition like 3% so model accuracy is likely to 97%
2. Softmax which turns training data into vectors that can be put onto a probability distrubition, so it helps to predict the outcome on a distributed by likelihood.
3. try other activation functions, increas volume of data, add more layers